# Baselines: ViT-Tiny and DeiT-Tiny (ImageNet-100)\nThis notebook is intentionally thin: it calls into `src/` modules.\n

In [ ]:
!pip -q install -r ../requirements.txt\n!pip -q install -e ..\n

In [ ]:
from src.imagenet_mapping import build_imagenet100_to_1k_map\nfrom src.models import ModelConfig, create_model, shrink_imagenet1k_head_to_imagenet100\nfrom src.data import DataConfig, load_imagenet100_split, build_transform_for_model, apply_timm_preprocess, build_loader\nfrom src.eval import evaluate_accuracy_latency_throughput, compute_gflops\nfrom src.utils import get_device, num_params\nimport torch\n

In [ ]:
device = get_device()\nmaps = build_imagenet100_to_1k_map()\n

In [ ]:
def run(model_id: str, batch_size: int = 64):\n    model = create_model(ModelConfig(model_id=model_id, pretrained=True))\n    model = shrink_imagenet1k_head_to_imagenet100(model, maps.new_to_old_map, num_classes=100)\n    model = model.to(device).eval()\n    ds = load_imagenet100_split(DataConfig(split='validation'))\n    transform = build_transform_for_model(model)\n    ds_t = apply_timm_preprocess(ds, transform)\n    loader = build_loader(ds_t, DataConfig(batch_size=batch_size, split='validation', shuffle=False))\n    metrics = evaluate_accuracy_latency_throughput(model, loader, device)\n    sample = ds_t[0]['pixel_values'].unsqueeze(0).to(device)\n    gflops = compute_gflops(model, sample)\n    return {\n        'model': model_id,\n        'params_m': num_params(model)/1e6,\n        'gflops': gflops,\n        **metrics\n    }\n

In [ ]:
run('vit_tiny_patch16_224')\n

In [ ]:
run('deit_tiny_patch16_224')\n